In [6]:

# Импорт библиотек
import pandas as pd
import duckdb

# Задание таблиц БД
users = pd.read_csv('users.csv')
course_users = pd.read_csv('course_users.csv')
courses = pd.read_csv('courses.csv')
course_types = pd.read_csv('course_types.csv')
lessons = pd.read_csv('lessons.csv')
subjects = pd.read_csv('subjects.csv')
cities = pd.read_csv('cities.csv')
homework_done = pd.read_csv('homework_done.csv')
homework = pd.read_csv('homework.csv')
homework_lessons = pd.read_csv('homework_lessons.csv')
user_roles = pd.read_csv('user_roles.csv') 

# Задание SQL-запроса
query = """
-- CTE: Подсчет количества сданных ДЗ на курсе
WITH homework_counts AS (
    SELECT
        cu.user_id,
        cu.course_id,
        COUNT(hd.id) AS homework_done_count
    FROM course_users cu
    INNER JOIN homework_done hd ON cu.user_id = hd.user_id -- присоединяем таблицу с выполненными ДЗ
    INNER JOIN homework_lessons hl ON hd.homework_id = hl.homework_id -- присоединяем таблицу для того чтобы по ДЗ определить к какому уроку оно относится
    INNER JOIN lessons l ON hl.lesson_id = l.id AND l.course_id = cu.course_id -- по уроку определяем к какому курсу относится дз
    GROUP BY cu.user_id, cu.course_id -- группируем для подсчета количества
)

-- Основной запрос
SELECT
    cu.course_id AS "ID курса",                          
    c.name AS "Название курса",                       
    s.name AS "Предмет",                                
    s.project AS "Тип предмета",                        
    ct.name AS "Тип курса",                             
    c.starts_at AS "Дата старта курса",                 
    u.id AS "ID ученика",                              
    u.last_name AS "Фамилия ученика",                   
    ci.name AS "Город ученика",                        
    cu.active AS "Ученик не отчислен с курса",          
    cu.created_at AS "Дата открытия курса ученику",     
    FLOOR(cu.available_lessons / c.lessons_in_month) 
        AS "Количество открытых месяцев курса у ученика",  -- По логике количество доступных ученику занятий по курсу / количество занятий в месяц по курсу, с округлением вниз
    COALESCE(hc.homework_done_count, 0) 
        AS "Число сданных ДЗ ученика на курсе"          -- Количество выполненных ДЗ (с проверкой на NULL)
FROM course_users cu
JOIN users u ON cu.user_id = u.id
JOIN courses c ON cu.course_id = c.id
JOIN subjects s ON c.subject_id = s.id
JOIN course_types ct ON c.course_type_id = ct.id
JOIN cities ci ON u.city_id = ci.id
LEFT JOIN homework_counts hc -- Присоединяем слева для того, чтобы не отсечь учеников без выполненных ДЗ 
    ON cu.user_id = hc.user_id AND cu.course_id = hc.course_id
WHERE ct.name = 'Годовой' -- Фильтр по типу курса
  -- AND cu.active = 1  -- Фильтр: только активные ученики
GROUP BY 
    cu.course_id, c.name, s.name, s.project, ct.name, c.starts_at, 
    u.id, u.last_name, ci.name, cu.active, cu.created_at, 
    cu.available_lessons, c.lessons_in_month, hc.homework_done_count;
"""
df = duckdb.query(query).to_df()


# Вывод результата
display(df)

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Количество открытых месяцев курса у ученика,Число сданных ДЗ ученика на курсе
0,18,Годовой 2к25 стандарт,История,ЕГЭ,Годовой,2024-09-18,10035,Ерохина,Ангарск,1,2024-08-11 13:25:28,3.0,16
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10051,Третьякова,Тотьма,1,2024-07-26 18:50:03,1.0,8
2,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10365,Лысова,Нижний Новгород,1,2024-07-30 20:01:27,1.0,7
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10374,Быкова,Москва,1,2024-09-19 10:57:47,2.0,8
4,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10414,Пищик,Краснодар,1,2024-08-22 18:38:01,2.0,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5380,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,10338,Иванова,Ачинск,0,2024-07-30 16:14:21,0.0,0
5381,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10661,Кулигин,Красноярск,0,2024-08-02 12:11:38,0.0,0
5382,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,30180,Могилева,Новосибирск,1,2024-10-06 16:02:35,2.0,0
5383,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,33254,Крачковская,Кемерово,0,2024-10-10 15:19:43,1.0,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5385 entries, 0 to 5384
Data columns (total 13 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   ID курса                                     5385 non-null   int64  
 1   Название курса                               5385 non-null   object 
 2   Предмет                                      5385 non-null   object 
 3   Тип предмета                                 5385 non-null   object 
 4   Тип курса                                    5385 non-null   object 
 5   Дата старта курса                            5385 non-null   object 
 6   ID ученика                                   5385 non-null   int64  
 7   Фамилия ученика                              5385 non-null   object 
 8   Город ученика                                5385 non-null   object 
 9   Ученик не отчислен с курса                   5385 non-null   int64  
 10  

#### 1.Проверим форматы данных

- Преобразуем строковые даты в datetime и столбец `Количество открытых месяцев курса у ученика` в целочисленное

In [11]:
df["Дата старта курса"] = pd.to_datetime(df["Дата старта курса"], errors="coerce")
df["Дата открытия курса ученику"] = pd.to_datetime(df["Дата открытия курса ученику"], errors="coerce")
df["Количество открытых месяцев курса у ученика"] = df["Количество открытых месяцев курса у ученика"].astype("Int64")

#### 2.Проверим данные на пропуски

- Посчитаем сумму пропущенных значений по столбцам и проанализируем их

In [14]:
df.isna().sum() #Вывод суммы пропусков по столбцам

ID курса                                        0
Название курса                                  0
Предмет                                         0
Тип предмета                                    0
Тип курса                                       0
Дата старта курса                               0
ID ученика                                      0
Фамилия ученика                                 0
Город ученика                                   0
Ученик не отчислен с курса                      0
Дата открытия курса ученику                     0
Количество открытых месяцев курса у ученика    15
Число сданных ДЗ ученика на курсе               0
dtype: int64

In [16]:
df_with_misses = df[df.isna().any(axis=1)] #вывод строк с пропущенными значениями 
df_with_misses

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Количество открытых месяцев курса у ученика,Число сданных ДЗ ученика на курсе
625,73,Годовой курс 2к25 комбо (биология + русский + ...,Русский ОГЭ,ОГЭ,Годовой,2024-09-22,30273,Петросова,Москва,1,2024-10-01 19:25:57,<NA>,0
654,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,28539,Зацепина,Тюмень,1,2024-09-25 13:33:15,<NA>,0
1272,74,Годовой курс 2к25 комбо (биология + обществозн...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-22,28327,Юшкова,Серов,1,2024-09-24 15:58:00,<NA>,0
1940,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,25975,Aix,Краснодар,1,2024-10-24 14:12:31,<NA>,0
1964,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,30135,Подгузова,Рязань,1,2024-10-01 14:27:08,<NA>,0
2622,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,25219,Лысов,Хабаровск,1,2024-09-16 13:45:20,<NA>,0
2678,74,Годовой курс 2к25 комбо (биология + обществозн...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-22,30965,Таськина,Липецк,1,2024-10-03 18:51:12,<NA>,0
3306,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,28200,Chukhno,Когалым,1,2024-09-24 09:05:46,<NA>,0
3325,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,28420,Губейдулин,Мытищи,1,2024-09-24 19:37:32,<NA>,0
3349,39,Годовой курс 2к25 комбо (русский + обществозна...,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-15,28914,Цып,Великие Луки,1,2024-09-26 21:12:10,<NA>,0


- Оценим ситуацию, проверим пропуски на предмет возникновения. Узнаем зависят ли они от курса, предмета, типа предмета или же пропуски вовсе случайны.

In [19]:
display(df_with_misses['ID курса'].value_counts()) # группировка пропусков по признакам
display(df_with_misses['Предмет'].value_counts())
display(df_with_misses['Тип предмета'].value_counts())

ID курса
39    11
74     3
73     1
Name: count, dtype: int64

Предмет
Обществознание ОГЭ    14
Русский ОГЭ            1
Name: count, dtype: int64

Тип предмета
ОГЭ    15
Name: count, dtype: int64

In [21]:
display(df['ID курса'].value_counts()) # Общее количество значений по признакам
display(df['Предмет'].value_counts())
display(df['Тип предмета'].value_counts())

ID курса
16    1652
19     856
22     426
17     391
23     380
18     355
25     271
24     167
36     155
38     151
37     133
21     129
41     108
20     104
42      48
39      22
56      14
59       9
57       5
74       3
79       2
62       1
73       1
81       1
80       1
Name: count, dtype: int64

Предмет
Обществознание        1668
Русский                865
Математика             488
Литература             427
Английский язык        397
История                356
Биология               271
Обществознание ОГЭ     180
Математика (база)      167
Математика ОГЭ         151
Русский ОГЭ            134
Химия                  129
Физика                 104
Биология ОГЭ            48
Name: count, dtype: int64

Тип предмета
ЕГЭ    4872
ОГЭ     513
Name: count, dtype: int64

Корреляции между курсом, предметом, типом предметом и пропусками не наблюдается, но можно сделать некоторые выводы, которые могут быть полезны специалистам по данным
- На курсе с ID = 39 , 11 из 22 учеников имеют пропуски в количестве доступных уроков
- На крусах 73 и 74, такая ситуация у всех обучающихся, но выборка мала для каких - либо выводов - суммарно 4 человека

Так как пропуски занимают незначительное количество от общего количества и по курсам 73 и 74 сложно будет получить информацию, которая требуется при дальнейшем анализе, то удалим строки с пропусками значений.

In [24]:
df = df.dropna()

#### 3. Проверка данных на наличие дубликатов и числовых аномалий

In [27]:
df.duplicated().sum()

0

In [29]:
print("\nУникальные значения в категориальных столбцах:")
for column in df.select_dtypes(include=['object']).columns: #Проходимся по столбцам с типом 'object' и выводим их уникальные значения
    print(f"\n{column}:")
    display(df[column].unique())


Уникальные значения в категориальных столбцах:

Название курса:


array(['Годовой 2к25 стандарт', 'В погоне за пятеркой 2К25 стандарт',
       'Годовой курс 2к25', 'Годовой 2к25 платинум',
       'Годовой 2К25 стандарт с Машей', 'Годовой 2К25 стандарт с Катей',
       'Годовой 2к25 минимал',
       'Годовой курс 2к25 комбо (русский + обществознание + математика)'],
      dtype=object)


Предмет:


array(['История', 'Русский', 'Обществознание', 'Английский язык',
       'Математика (база)', 'Литература', 'Биология',
       'Обществознание ОГЭ', 'Математика', 'Биология ОГЭ', 'Химия',
       'Математика ОГЭ', 'Русский ОГЭ', 'Физика'], dtype=object)


Тип предмета:


...

array(['Годовой'], dtype=object)


Фамилия ученика:


array(['Ерохина', 'Третьякова', 'Лысова', ..., 'Башина', 'Козырева',
       'Хоперскова'], dtype=object)


Город ученика:


array(['Ангарск', 'Тотьма', 'Нижний Новгород', 'Москва', 'Краснодар',
       'Казань', 'Калуга', 'Республика Дагестанские Огни', 'Владивосток',
       'Кодинск', 'Йошкар-Ола', 'Подольск', 'Брянск', 'Красноярск',
       'Ханты-Мансийск', 'Санкт-Петербург', 'Олонец', 'Тверь', 'Тула',
       'Волгоград', 'Липецк', 'Ноябрьск', 'Одинцово', 'Ростов-на-Дону',
       'Ставрополь', 'Бирск', 'Ртищево', 'Апшеронск', 'Избербаш',
       'Иркутск', 'Кемерово', 'Новосибирск', 'Люберцы', 'Раменское',
       'Всеволожск', 'Архангельск', 'Ельня', 'Сергиев Посад',
       'Владикавказ', 'Видное', 'Ковров', 'Прокопьевск', 'Ейск', 'Самара',
       'Киров', 'Каменск-Уральский', 'Кропоткин', 'Барнаул', 'Ижевск',
       'Верхняя Салда', 'Хабаровск', 'Норильск', 'Моршанск', 'Орёл',
       'Балтийск', 'Ессентуки', 'Канаш', 'Альметьевск', 'Темрюк',
       'Орехово-Зуево', 'Ахтубинск', 'Минеральные Воды', 'Тольятти',
       'Омск', 'Новокузнецк', 'Северодвинск', 'Домодедово', 'Тамбов',
       'Вязники', 'Тосно', '

Полные дубликаты, а также неявные дубликаты в категориальных столбцах отсутствуют

In [32]:
df.describe() # Описательная статистика по числовым столбцам

,ID курса,Дата старта курса,ID ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Количество открытых месяцев курса у ученика,Число сданных ДЗ ученика на курсе
count,5370.000000,5370,5370.000000,5370.000000,5370,5370.0,5370.00000
mean,21.324022,2024-09-19 04:59:31.843575552,24584.787151,0.943389,2024-09-17 20:59:51.596089600,2.216574,10.31676
min,16.000000,2024-09-15 00:00:00,10001.000000,0.000000,2024-07-25 18:04:20,0.0,0.00000
25%,16.000000,2024-09-16 00:00:00,10980.250000,1.000000,2024-09-16 12:44:47,2.0,5.00000
50%,19.000000,2024-09-17 00:00:00,28293.000000,1.000000,2024-09-25 19:32:34,2.0,10.00000
75%,23.000000,2024-09-21 00:00:00,30577.000000,1.000000,2024-10-06 09:13:21,2.0,15.00000
max,81.000000,2024-10-15 00:00:00,33999.000000,1.000000,2024-11-09 18:44:36,10.0,34.00000
std,7.401306,NaN,8651.624755,0.231119,NaN,1.722136,6.92592


Аномалии в числовых значениях остуствуют

#### 4.Распределим учеников по волнам

- Создадим столбец `Волна`, который будет рассчитываться на основе разницы дней дат открытия курса ученик и старта курса.
- К этой разнице прибавим 6 и целочисленно поделим на 7, что и будет давать нам номер волны.
- Также ограничимся минимальными значениями в 0 и 5 и добавим в конец строку 'Волна'

In [36]:
df['Волна'] = (((df['Дата открытия курса ученику'] - df['Дата старта курса']).dt.days + 6) // 7).clip(lower=0, upper=5).astype(str) + ' волна'

In [38]:
df

,ID курса,Название курса,Предмет,Тип предмета,Тип курса,Дата старта курса,ID ученика,Фамилия ученика,Город ученика,Ученик не отчислен с курса,Дата открытия курса ученику,Количество открытых месяцев курса у ученика,Число сданных ДЗ ученика на курсе,Волна
0,18,Годовой 2к25 стандарт,История,ЕГЭ,Годовой,2024-09-18,10035,Ерохина,Ангарск,1,2024-08-11 13:25:28,3,16,0 волна
1,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10051,Третьякова,Тотьма,1,2024-07-26 18:50:03,1,8,0 волна
2,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10365,Лысова,Нижний Новгород,1,2024-07-30 20:01:27,1,7,0 волна
3,16,Годовой 2к25 стандарт,Обществознание,ЕГЭ,Годовой,2024-09-16,10374,Быкова,Москва,1,2024-09-19 10:57:47,2,8,1 волна
4,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10414,Пищик,Краснодар,1,2024-08-22 18:38:01,2,14,0 волна
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5380,37,Годовой курс 2к25,Русский ОГЭ,ОГЭ,Годовой,2024-09-25,10338,Иванова,Ачинск,0,2024-07-30 16:14:21,0,0,0 волна
5381,19,Годовой 2к25 стандарт,Русский,ЕГЭ,Годовой,2024-09-21,10661,Кулигин,Красноярск,0,2024-08-02 12:11:38,0,0,0 волна
5382,24,В погоне за пятеркой 2К25 стандарт,Математика (база),ЕГЭ,Годовой,2024-09-17,30180,Могилева,Новосибирск,1,2024-10-06 16:02:35,2,0,3 волна
5383,36,Годовой курс 2к25,Обществознание ОГЭ,ОГЭ,Годовой,2024-09-30,33254,Крачковская,Кемерово,0,2024-10-10 15:19:43,1,0,2 волна
